In [262]:
import nltk
import random
import itertools
from collections import defaultdict
from tqdm import tqdm
from nltk import CFG
from time import sleep

In [263]:
meta_grammar = [
'S -> RULE',
'RULE -> NONTERMINAL " -> " CONTENT',
'CONTENT -> CONTENT " | " CONTENT | TERMINAL_PRE | TERMINAL_PRE " " NONTERMINAL | NONTERMINAL " " TERMINAL_PRE | NONTERMINAL " " NONTERMINAL | TERMINAL_PRE " " TERMINAL_PRE | TERMINAL_PRE " " NONTERMINAL " " TERMINAL_PRE',
'TERMINAL_PRE -> "#" TERMINAL "#" ',    # "#" gets replaced with double quotes later
# 'TERMINAL -> "a" | "b" | "c" | "d" | "e" | "f" | "g" | "h" | "i" | "j" | "k" | "l" | "m" | "n" | "o" | "p" | "q" | "r" | "s" | "t" | "u" | "v" | "w" | "x" | "y" | "z"',
# 'NONTERMINAL -> "A" | "B" | "C" | "D" | "E" | "F" | "G" | "H" | "I" | "J" | "K" | "L" | "M" | "N" | "O" | "P" | "Q" | "R" | "S" | "T" | "U" | "V" | "W" | "X" | "Y" | "Z"',
]

In [264]:
# generate a random valid sentence from the grammar
def generate_random_sentence(grammar, join_char=' '):

    # how often to try to generate a valid sentence
    # if we fail, we raise a ValueError
    max_tries = 5

    # how often to try to expand a non-terminal
    # after this many iterations, we give up and try again
    max_iterations = 10000

    # Ensure the grammar is an CFG object
    if isinstance(grammar, str) or isinstance(grammar, list):
        grammar = CFG.fromstring(grammar)
    elif not isinstance(grammar, CFG):
        raise ValueError("The grammar must be a string or an CFG object.")

    for current_try in range(max_tries):

        current_iteration = 0
        sentence = [grammar.start()]

        # as long as there are non-terminals in the sentence
        while any(nltk.grammar.is_nonterminal(symbol) for symbol in sentence):

            # find the non-terminals
            non_terminals = [i for i, symbol in enumerate(sentence) if nltk.grammar.is_nonterminal(symbol)]

            if not non_terminals:
                break  # we are done, there are only non-terminals in the sentence

            # randomly choose a non-terminal to expand
            nt_index = random.choice(non_terminals)
            symbol = sentence[nt_index]

            # randomly choose a production for the non-terminal
            productions = grammar.productions(lhs=symbol)
            
            production = random.choice(productions)

            # replace the non-terminal with the production
            sentence = sentence[:nt_index] + list(production.rhs()) + sentence[nt_index+1:]

            # avoid infinite loops
            current_iteration += 1
            if current_iteration > max_iterations:
                break
        
        # if the sentence is valid, return it
        if not any(nltk.grammar.is_nonterminal(symbol) for symbol in sentence):
            return join_char.join(str(symbol) for symbol in sentence)

    raise ValueError("The grammar is too complex to generate a valid sentence.")

In [265]:
# this function generates a random grammar
def generate_random_grammar(terminals, nonterminals, n_rules=5):
    assert n_rules >= len(nonterminals), "There must be at least as many rules as nonterminals (n_rules >= len(nonterminals))"

    meta_rules = meta_grammar.copy()

    # generate rules for terminals
    for terminal in terminals:
        rule = f'TERMINAL -> "{terminal}"'
        meta_rules.append(rule)
    
    # generate rules for nonterminals
    for nonterminal in nonterminals:
        rule = f'NONTERMINAL -> "{nonterminal}"'
        meta_rules.append(rule)

    # generate rules
    random_rules = []

    # the first n_nonterminal rules start with the nonterminals
    # so that every nonterminal has at least one rule
    for nonterminal in nonterminals:
        # generate one CONTENT and build the rule like this:
        random_content = generate_random_sentence(meta_rules, join_char='')
        # replace the first character of the content with the nonterminal
        random_rule = nonterminal + random_content[1:]

        # replace all '#' with '""
        random_rule = random_rule.replace('#', '"')

        random_rules.append(random_rule)

    while len(random_rules) < n_rules:
        random_rule = generate_random_sentence(meta_rules, join_char='')

        if random_rule not in random_rules:
            random_rule = random_rule.replace('#', '"')
            random_rules.append(random_rule)
    
    # sort the rules
    random_rules.sort()

    # add the starting rule to the beginning of the list
    starting_rule = 'S -> ' + nonterminals[0]
    random_rules.insert(0, starting_rule)
    
    # convert the rules to a string
    random_rules = '\n'.join(random_rules)

    return random_rules

In [266]:
# checks if a sentence is in the grammar
def sentence_in_grammar(sentence: str, grammar) -> bool:
    
    # Ensure the grammar is an CFG object
    if isinstance(grammar, str) or isinstance(grammar, list):
        grammar = CFG.fromstring(grammar)
    elif not isinstance(grammar, CFG):
        raise ValueError("The grammar must be a string or an CFG object.")
    
    # split the sentence into a char array
    tokens = list(sentence)
    
    # Initialize the parser with the given grammar
    parser = nltk.parse.EarleyChartParser(grammar)
    
    # Attempt to parse the tokenized sentence
    try:
        # Generate all possible parse trees for the sentence
        for parse in parser.parse(tokens):
            # If at least one parse tree is found, the sentence can be generated by the grammar
            return True
    except ValueError as e:
        # Catch and handle the case where the sentence contains tokens not in the grammar
        return False
    
    # If no parse trees are found, the sentence cannot be generated by the grammar
    return False

In [267]:
# a generator that generates all valid strings for a given grammar until a given length
def generate_valid_strings(terminals, grammar, max_length=5):
    
    # Ensure the grammar is an CFG object
    if isinstance(grammar, str) or isinstance(grammar, list):
        grammar = CFG.fromstring(grammar)
    elif not isinstance(grammar, CFG):
        raise ValueError("The grammar must be a string or an CFG object.")

    # Iterate over all lengths of strings
    for length in range(1, max_length + 1):
        # Generate all combinations with repetition of the current length
        for word in itertools.product(terminals, repeat=length):
            # Join the characters to form a string
            sentence = ''.join(word)

            # Check if the sentence is in the grammar
            in_grammar = sentence_in_grammar(sentence, grammar)

            # If the sentence is in the grammar, yield it
            if in_grammar:
                yield sentence

In [268]:
# generate a list of uppercase letters
def generate_nonterminals(n: int):
    return [chr(65+i) for i in range(n)]

# generate a list lowercase letters
def generate_terminals(n: int):
    return [chr(97+i) for i in range(n)]

In [269]:
# returns true, if the grammar will loop endlessly
# implemented by checking whether the graph of the rules is transient
def build_transition_graph(grammar):
    """Build a transition graph from the grammar for Markov chain analysis."""
    graph = defaultdict(list)
    for production in grammar.productions():
        lhs = str(production.lhs())
        rhs_symbols = [str(rhs) for rhs in production.rhs() if nltk.grammar.is_nonterminal(rhs)]
        graph[lhs].extend(rhs_symbols)
    return graph

def find_reachable_states(graph, start='S'):
    """Find all states reachable from the start symbol using DFS."""
    reachable = set()
    stack = [start]
    while stack:
        current = stack.pop()
        if current not in reachable:
            reachable.add(current)
            stack.extend(graph[current])
    return reachable

def find_absorbing_states(grammar):
    """Identify terminal symbols (absorbing states) in the grammar."""
    return {str(production.lhs()) for production in grammar.productions() if all(not nltk.grammar.is_nonterminal(rhs) for rhs in production.rhs())}

def is_grammar_transient(grammar_input):
    """Check if the grammar is transient using Markov chain analysis."""
    if isinstance(grammar_input, str) or isinstance(grammar_input, list):
        grammar = CFG.fromstring('\n'.join(grammar_input) if isinstance(grammar_input, list) else grammar_input)
    elif not isinstance(grammar_input, CFG):
        raise ValueError("The grammar must be a string or an CFG object.")
    
    graph = build_transition_graph(grammar)
    absorbing_states = find_absorbing_states(grammar)
    reachable_states = find_reachable_states(graph)
    
    # Initialize all non-terminals as transient until proven otherwise
    transient = {node: True for node in graph}
    
    # Markov chain analysis: Check reachability of absorbing states
    for start in graph:
        visited = set()
        stack = [start]

        # check if start is a reachable state
        # if not, we will never generate it
        # we can ignore it and give it the value True
        if start not in reachable_states:
            transient[start] = True
            continue

        while stack:
            current = stack.pop()
            if current in absorbing_states:
                break  # Found path to absorbing state
            if current not in visited:
                visited.add(current)
                stack.extend(graph[current])
        else:
            transient[start] = False  # No path to absorbing state found
    
    return all(transient.values())

In [270]:
# True, despite B and C loop, S can reach terminals through A and D
mixed_starting_nonterminals = [
    'S -> A | D',
    'A -> "a"',
    'B -> C',
    'C -> B',
    'D -> "d"',
]

grammar = CFG.fromstring('\n'.join(mixed_starting_nonterminals))
graph = build_transition_graph(grammar)
reachable_states = find_reachable_states(graph, start='S')
absorbing_states = find_absorbing_states(grammar)

print("Graph:", graph)
print("Reachable states:", reachable_states)
print("Absorbing states:", absorbing_states)
print("Is transient:", is_grammar_transient(mixed_starting_nonterminals))

Graph: defaultdict(<class 'list'>, {'S': ['A', 'D'], 'A': [], 'B': ['C'], 'C': ['B'], 'D': []})
Reachable states: {'D', 'S', 'A'}
Absorbing states: {'D', 'A'}
Is transient: True


In [271]:
# Test cases for is_grammar_transient
# True, simple path to terminal
transient_grammar = [
    'S -> A',
    'A -> B',
    'B -> "c"',
]

# False, loop includes A again
nontransient_grammar = [
    'S -> A',
    'A -> B',
    'B -> "c" A',
]

# True, A has an alternative path to "b"
direct_loop_with_terminal = [
    'S -> A',
    'A -> A | "b"',
]

# False, indirect loop without terminal resolution
indirect_loop_without_terminal = [
    'S -> A',
    'A -> B',
    'B -> C',
    'C -> A',
]

# True, despite B and C loop, S can reach terminals through A and D
mixed_starting_nonterminals = [
    'S -> A | D',
    'A -> "a"',
    'B -> C',
    'C -> B',
    'D -> "d"',
]

# True, loops exist but all have escape to terminals
nested_loops_with_escape = [
    'S -> A',
    'A -> B | "e"',
    'B -> C',
    'C -> D | B',
    'D -> "f"',
]

# True, deep nesting but eventually reaches terminal
deeply_nested_structure = [
    'S -> A',
    'A -> B',
    'B -> C',
    'C -> D',
    'D -> E',
    'E -> "g"',
]

# True, complex with multiple paths, some loops, some lead to terminals
complex_grammar_multiple_paths = [
    'S -> A | X',
    'A -> B | "h"',
    'B -> C | "i"',
    'C -> A | "j"',
    'X -> Y',
    'Y -> Z | X',
    'Z -> "k"',
]

                                                            # Expected output
print(is_grammar_transient(transient_grammar))              # True
print(is_grammar_transient(nontransient_grammar))           # False
print(is_grammar_transient(direct_loop_with_terminal))      # True
print(is_grammar_transient(indirect_loop_without_terminal)) # False
print(is_grammar_transient(mixed_starting_nonterminals))    # True
print(is_grammar_transient(nested_loops_with_escape))       # True
print(is_grammar_transient(deeply_nested_structure))        # True
print(is_grammar_transient(complex_grammar_multiple_paths)) # True

True
False
True
False
True
True
True
True


In [229]:
def word_table(terminals, grammar, max_length=3):
    
    # Ensure the grammar is an CFG object
    if isinstance(grammar, str) or isinstance(grammar, list):
        grammar = CFG.fromstring(grammar)
    elif not isinstance(grammar, CFG):
        raise ValueError("The grammar must be a string or an CFG object.")

    # Iterate over all lengths of strings
    for current_length in range(1, max_length+1):
        # Generate all combinations with repetition of the current length
        for word in itertools.product(terminals, repeat=current_length):
            # Join the characters to form a string
            sentence = ''.join(word)

            # Check if the sentence is in the grammar
            in_grammar = sentence_in_grammar(sentence, grammar)

            # Print the sentence and whether it's in the grammar
            print(f"{sentence}: {in_grammar}")

In [299]:
# these parameters control the grammar
n_nonterminals = 3
n_terminals = 5
n_rules = 8

# generate the nonterminals and terminals
nonterminals = generate_nonterminals(n_nonterminals)
terminals = "🟥🟦🟨🟩🟧🟪🟫⬛⬜🔴🔵🟡🟢🟠🟣🟤⚫⚪"
terminals = list(terminals)

terminals = terminals[:n_terminals]

grammar = generate_random_grammar(terminals, nonterminals, n_rules=n_rules)
is_transient = is_grammar_transient(grammar)

print(grammar)
print(f'is_transient: {is_transient}')

if is_transient:
    for i in range(10):
        sentence = generate_random_sentence(grammar, join_char='')
        print(sentence)
else:
    print("The grammar is not transient, so it will loop endlessly.")

S -> A
A -> "🟦" "🟥"
A -> "🟨" C
B -> "🟥" "🟧"
B -> C B
C -> "🟥" "🟨"
C -> "🟦" C
C -> A "🟨" | "🟨" "🟧"
C -> A B
is_transient: True
🟨🟥🟨
🟦🟥
🟦🟥
🟦🟥
🟦🟥
🟨🟦🟥🟨
🟦🟥
🟦🟥
🟨🟥🟨
🟨🟨🟧


In [231]:
# try all strings and print whether they are in the grammar
word_table(terminals, grammar=grammar, max_length=3)

🟥: True
🟦: False
🟨: False
🟩: False
🟧: True
🟥🟥: False
🟥🟦: False
🟥🟨: False
🟥🟩: False
🟥🟧: False
🟦🟥: False
🟦🟦: False
🟦🟨: False
🟦🟩: False
🟦🟧: False
🟨🟥: False
🟨🟦: False
🟨🟨: False
🟨🟩: False
🟨🟧: False
🟩🟥: False
🟩🟦: False
🟩🟨: False
🟩🟩: False
🟩🟧: False
🟧🟥: False
🟧🟦: False
🟧🟨: False
🟧🟩: False
🟧🟧: False
🟥🟥🟥: False
🟥🟥🟦: False
🟥🟥🟨: False
🟥🟥🟩: False
🟥🟥🟧: False
🟥🟦🟥: False
🟥🟦🟦: False
🟥🟦🟨: False
🟥🟦🟩: False
🟥🟦🟧: False
🟥🟨🟥: False
🟥🟨🟦: False
🟥🟨🟨: False
🟥🟨🟩: False
🟥🟨🟧: False
🟥🟩🟥: False
🟥🟩🟦: False
🟥🟩🟨: False
🟥🟩🟩: False
🟥🟩🟧: False
🟥🟧🟥: False
🟥🟧🟦: False
🟥🟧🟨: False
🟥🟧🟩: False
🟥🟧🟧: False
🟦🟥🟥: False
🟦🟥🟦: False
🟦🟥🟨: False
🟦🟥🟩: False
🟦🟥🟧: False
🟦🟦🟥: False
🟦🟦🟦: False
🟦🟦🟨: False
🟦🟦🟩: False
🟦🟦🟧: False
🟦🟨🟥: False
🟦🟨🟦: False
🟦🟨🟨: False
🟦🟨🟩: False
🟦🟨🟧: False
🟦🟩🟥: False
🟦🟩🟦: False
🟦🟩🟨: False
🟦🟩🟩: False
🟦🟩🟧: False
🟦🟧🟥: False
🟦🟧🟦: False
🟦🟧🟨: False
🟦🟧🟩: False
🟦🟧🟧: False
🟨🟥🟥: False
🟨🟥🟦: False
🟨🟥🟨: False
🟨🟥🟩: False
🟨🟥🟧: False
🟨🟦🟥: False
🟨🟦🟦: False
🟨🟦🟨: False
🟨🟦🟩: False
🟨🟦🟧: False
🟨🟨🟥: False
🟨🟨🟦: False
🟨🟨🟨: False
🟨🟨🟩: False
🟨🟨🟧

In [234]:
# print all valid strings of length 5
for sentence in generate_valid_strings(terminals, grammar, max_length=5):
    print(sentence)

🟥
🟧
🟩🟩🟥
🟧🟥🟧
🟧🟧🟧
🟥🟩🟩🟥
🟩🟩🟩🟥
🟩🟩🟩🟧
🟧🟩🟩🟥
🟥🟥🟩🟩🟥
🟥🟩🟩🟩🟥
🟥🟩🟩🟩🟧
🟥🟧🟩🟩🟥
🟧🟥🟩🟩🟥
🟧🟩🟩🟥🟧
🟧🟩🟩🟩🟥
🟧🟩🟩🟩🟧
🟧🟧🟥🟧🟧
🟧🟧🟩🟩🟥
🟧🟧🟧🟧🟧


In [233]:
# write 1.000.000 sentences to a file
if False:
    with open('sentences.txt', 'w') as f:
        for _ in tqdm(range(1_000_000)):
            f.write(generate_random_sentence(grammar, '') + '\n')